In [9]:

import pandas as pd
import numpy as np
from keras import models,backend,optimizers,layers
from keras.models import Model
from keras.layers import LSTM,Dense,Dropout,concatenate,Input,GRU,SimpleRNN
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import preprocessing
import sys
import sklearn.externals 
import joblib
from keras.callbacks import EarlyStopping
from math import sqrt
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
def mae_f( y_v , p_v , d ):
    y_v[:,d]=y_v[:,d].astype(float)

    return sum(abs(y_v[:,d]-p_v[:,d]))/len(y_v)

def mse_f( y_v , p_v , d ):
    y_v[:,d]=y_v[:,d].astype(float)

    return sum((y_v[:,d]-p_v[:,d])*(y_v[:,d]-p_v[:,d]))/len(y_v)

def mape_f( y_v , p_v , d ):
    y_v[:,d]=y_v[:,d].astype(float)
    return (1/len(y_v))*(sum(abs((p_v[:,d]-y_v[:,d])/y_v[:,d])))

def rmse_f( y_v , p_v , d ):
    y_v[:,d]=y_v[:,d].astype(float)
    return sqrt(sum((y_v[:,d]-p_v[:,d])*(y_v[:,d]-p_v[:,d]))/len(y_v))
#data_loc= pd.read_csv('new0.csv', encoding='utf-8')

#data_loc
data= pd.read_csv('D:/download/完成/final3.csv', encoding='utf-8')
#data_loc=data_loc.dropna()



In [10]:
#data_loc= pd.read_csv('D:/download/完成/final1.csv', encoding='utf-8')
#data_loc=data[data['location']==0]

data=data.dropna()
#data_loc=data_loc.drop(columns=['location','deviceId'])
validation_sample=80000
data = shuffle(data) #資料亂序

#col_name = data_loc.columns.tolist()
#for col in col_name:
    #drop_id = data_loc[data_loc[col]>=100].index 
    #data_loc=data_loc.drop(drop_id)
    #drop_id = data_loc[data_loc[col]==0].index 
    #data_loc=data_loc.drop(drop_id)

#=============資料處理===============#
loc_0_x=data.iloc[:,:24]
loc_0_x=loc_0_x.values
StandardScaler_scaler = preprocessing.StandardScaler()
StandardScaler_scaler = StandardScaler_scaler.fit(loc_0_x) #標準化模型
loc_0_x = StandardScaler_scaler.transform(loc_0_x)
joblib.dump(StandardScaler_scaler, 'scaler3_8.pkl')

['scaler3_8.pkl']

In [2]:
data_loc=data[data['location']==4]
data_loc

,deviceId,location,past_24_pm2.5,past_24_humi,past_24_temp,past_23_pm2.5,past_23_humi,past_23_temp,past_22_pm2.5,past_22_humi,...,15hrs,16hrs,17hrs,18hrs,19hrs,20hrs,21hrs,22hrs,23hrs,24hrs
62450,6840396964,4,13.64,76.23,13.70,16.29,78.83,13.39,22.11,79.38,...,29.00,27.41,22.64,27.94,23.70,22.11,19.99,14.70,23.70,14.70
62451,6840396964,4,16.29,78.83,13.39,22.11,79.38,13.37,18.40,79.48,...,27.41,22.64,27.94,23.70,22.11,19.99,14.70,23.70,14.70,14.17
62452,6840396964,4,22.11,79.38,13.37,18.40,79.48,13.29,14.70,80.16,...,22.64,27.94,23.70,22.11,19.99,14.70,23.70,14.70,14.17,18.40
62453,6840396964,4,18.40,79.48,13.29,14.70,80.16,13.66,21.05,73.94,...,27.94,23.70,22.11,19.99,14.70,23.70,14.70,14.17,18.40,17.34
62454,6840396964,4,14.70,80.16,13.66,21.05,73.94,15.74,14.70,69.06,...,23.70,22.11,19.99,14.70,23.70,14.70,14.17,18.40,17.34,11.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796603,7524224279,4,49.00,77.40,19.80,46.00,78.90,19.20,49.00,78.40,...,4.00,4.00,4.00,5.00,5.00,4.00,5.00,8.00,15.00,14.00
796604,7524224279,4,46.00,78.90,19.20,49.00,78.40,19.40,50.00,79.40,...,4.00,4.00,5.00,5.00,4.00,5.00,8.00,15.00,14.00,15.00
796605,7524224279,4,49.00,78.40,19.40,50.00,79.40,19.20,57.00,85.70,...,4.00,5.00,5.00,4.00,5.00,8.00,15.00,14.00,15.00,10.00
796606,7524224279,4,50.00,79.40,19.20,57.00,85.70,18.70,65.00,87.20,...,5.00,5.00,4.00,5.00,8.00,15.00,14.00,15.00,10.00,13.00


In [3]:
data_loc=data_loc.drop(data_loc.iloc[:,:50],axis=1)
data_loc=data_loc.drop(data_loc.iloc[:,32:48],axis=1)
data_loc

,past_8_pm2.5,past_8_humi,past_8_temp,past_7_pm2.5,past_7_humi,past_7_temp,past_6_pm2.5,past_6_humi,past_6_temp,past_5_pm2.5,...,past_1_humi,past_1_temp,1hrs,2hrs,3hrs,4hrs,5hrs,6hrs,7hrs,8hrs
62450,25.82,73.06,18.71,27.41,74.90,17.52,23.70,77.17,16.55,20.52,...,80.61,15.48,17.34,13.64,13.11,15.76,15.23,13.11,14.17,10.99
62451,27.41,74.90,17.52,23.70,77.17,16.55,20.52,77.82,16.03,21.58,...,79.06,15.78,13.64,13.11,15.76,15.23,13.11,14.17,10.99,11.52
62452,23.70,77.17,16.55,20.52,77.82,16.03,21.58,80.72,15.48,15.23,...,78.63,16.01,13.11,15.76,15.23,13.11,14.17,10.99,11.52,16.29
62453,20.52,77.82,16.03,21.58,80.72,15.48,15.23,79.91,15.60,16.29,...,78.50,16.04,15.76,15.23,13.11,14.17,10.99,11.52,16.29,14.70
62454,21.58,80.72,15.48,15.23,79.91,15.60,16.29,81.06,15.17,19.99,...,78.69,16.25,15.23,13.11,14.17,10.99,11.52,16.29,14.70,16.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796603,6.00,84.70,20.40,8.00,81.30,21.60,7.00,82.80,21.60,6.00,...,83.80,21.00,14.00,17.00,14.00,18.00,18.00,15.00,12.00,12.00
796604,8.00,81.30,21.60,7.00,82.80,21.60,6.00,77.40,22.70,9.00,...,84.30,20.50,17.00,14.00,18.00,18.00,15.00,12.00,12.00,13.00
796605,7.00,82.80,21.60,6.00,77.40,22.70,9.00,82.80,21.60,8.00,...,84.70,20.40,14.00,18.00,18.00,15.00,12.00,12.00,13.00,16.00
796606,6.00,77.40,22.70,9.00,82.80,21.60,8.00,82.80,21.00,8.00,...,86.70,19.80,18.00,18.00,15.00,12.00,12.00,13.00,16.00,15.00


In [9]:



loc_0_x=loc_0_x.reshape(len(loc_0_x),8,3) #作三維資料

loc_0_y=data_loc.iloc[:,24:].values

#=============測試集==============#
x_v=loc_0_x[validation_sample:]
y_v=loc_0_y[validation_sample:]

#=============訓練集資料===============#
x=loc_0_x[:validation_sample]
y=loc_0_y[:validation_sample]

#隨機分訓練集及測試集

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

  

#建立模型
input_data=Input(shape=(8,3))
m=Bidirectional(LSTM(256,return_sequences=True))(input_data)
m=Dropout(0.3)(m)
m=LSTM(256)(m)
m=Dropout(0.3)(m)
#m=LSTM(256)(m)
#m=Dropout(0.2)(m)

m=Dense(256,activation='relu')(m)
m=Dropout(0.3)(m)
m=Dense(256,activation='relu')(m)
m=Dropout(0.3)(m)
m=Dense(256,activation='relu')(m)
m=Dropout(0.3)(m)
output=Dense(8,activation=None)(m)

model = Model(inputs=input_data,outputs=output)
callback = EarlyStopping(monitor="val_loss", patience=100, verbose=1, mode="auto")
model.compile(optimizer='adam',loss=['mse'],metrics=['mae','mape'])
result = model.fit(X_train,y_train,epochs=1500 ,batch_size=2400,validation_data=(X_test, y_test),callbacks=[callback])
#model.save('D:/download/finalmodel/model0.h5')


Train on 56000 samples, validate on 24000 samples
Epoch 1/1500
56000/56000 [==============================] - 10s 173us/step - loss: 506.6171 - mean_absolute_error: 17.8597 - mean_absolute_percentage_error: 71.2027 - val_loss: 188.8602 - val_mean_absolute_error: 9.6482 - val_mean_absolute_percentage_error: 36.2280
Epoch 2/1500
56000/56000 [==============================] - 6s 110us/step - loss: 156.7020 - mean_absolute_error: 9.0917 - mean_absolute_percentage_error: 41.3257 - val_loss: 122.1652 - val_mean_absolute_error: 7.5231 - val_mean_absolute_percentage_error: 31.9106
Epoch 3/1500
56000/56000 [==============================] - 6s 110us/step - loss: 131.9844 - mean_absolute_error: 8.2493 - mean_absolute_percentage_error: 37.2203 - val_loss: 111.5705 - val_mean_absolute_error: 7.2004 - val_mean_absolute_percentage_error: 31.6733
Epoch 4/1500
56000/56000 [==============================] - 6s 110us/step - loss: 123.4484 - mean_absolute_error: 7.9414 - mean_absolute_percentage_error: 3

Epoch 32/1500
56000/56000 [==============================] - 7s 121us/step - loss: 82.7971 - mean_absolute_error: 6.1677 - mean_absolute_percentage_error: 27.1953 - val_loss: 85.0626 - val_mean_absolute_error: 5.9766 - val_mean_absolute_percentage_error: 23.7885
Epoch 33/1500
56000/56000 [==============================] - 7s 121us/step - loss: 81.4644 - mean_absolute_error: 6.1216 - mean_absolute_percentage_error: 27.0306 - val_loss: 81.7155 - val_mean_absolute_error: 5.8701 - val_mean_absolute_percentage_error: 23.7492
Epoch 34/1500
56000/56000 [==============================] - 7s 121us/step - loss: 81.7674 - mean_absolute_error: 6.1244 - mean_absolute_percentage_error: 27.0376 - val_loss: 83.7217 - val_mean_absolute_error: 5.9105 - val_mean_absolute_percentage_error: 23.6001
Epoch 35/1500
56000/56000 [==============================] - 7s 122us/step - loss: 80.6721 - mean_absolute_error: 6.0763 - mean_absolute_percentage_error: 26.8993 - val_loss: 88.6223 - val_mean_absolute_error: 6

56000/56000 [==============================] - 7s 125us/step - loss: 58.3366 - mean_absolute_error: 4.9721 - mean_absolute_percentage_error: 21.5244 - val_loss: 63.2632 - val_mean_absolute_error: 4.8515 - val_mean_absolute_percentage_error: 19.2639
Epoch 95/1500
56000/56000 [==============================] - 7s 125us/step - loss: 58.0333 - mean_absolute_error: 4.9610 - mean_absolute_percentage_error: 21.4427 - val_loss: 60.9361 - val_mean_absolute_error: 4.7245 - val_mean_absolute_percentage_error: 19.0181
Epoch 96/1500
56000/56000 [==============================] - 7s 126us/step - loss: 57.8847 - mean_absolute_error: 4.9442 - mean_absolute_percentage_error: 21.2901 - val_loss: 58.9392 - val_mean_absolute_error: 4.6276 - val_mean_absolute_percentage_error: 19.2201
Epoch 97/1500
56000/56000 [==============================] - 7s 126us/step - loss: 57.7181 - mean_absolute_error: 4.9432 - mean_absolute_percentage_error: 21.3060 - val_loss: 62.7264 - val_mean_absolute_error: 4.8229 - val_me

56000/56000 [==============================] - 7s 127us/step - loss: 46.6926 - mean_absolute_error: 4.3907 - mean_absolute_percentage_error: 18.7235 - val_loss: 53.6729 - val_mean_absolute_error: 4.2854 - val_mean_absolute_percentage_error: 17.6370
Epoch 157/1500
56000/56000 [==============================] - 7s 126us/step - loss: 46.6756 - mean_absolute_error: 4.3919 - mean_absolute_percentage_error: 18.7692 - val_loss: 54.4484 - val_mean_absolute_error: 4.3289 - val_mean_absolute_percentage_error: 17.8053
Epoch 158/1500
56000/56000 [==============================] - 7s 127us/step - loss: 46.1957 - mean_absolute_error: 4.3708 - mean_absolute_percentage_error: 18.6804 - val_loss: 53.9974 - val_mean_absolute_error: 4.3077 - val_mean_absolute_percentage_error: 17.6833
Epoch 159/1500
56000/56000 [==============================] - 7s 126us/step - loss: 45.8599 - mean_absolute_error: 4.3521 - mean_absolute_percentage_error: 18.5984 - val_loss: 53.5336 - val_mean_absolute_error: 4.2766 - val

56000/56000 [==============================] - 7s 127us/step - loss: 40.0704 - mean_absolute_error: 4.1000 - mean_absolute_percentage_error: 17.5038 - val_loss: 53.1959 - val_mean_absolute_error: 4.2268 - val_mean_absolute_percentage_error: 17.4393
Epoch 219/1500
56000/56000 [==============================] - 7s 126us/step - loss: 39.4430 - mean_absolute_error: 4.0919 - mean_absolute_percentage_error: 17.5005 - val_loss: 51.7650 - val_mean_absolute_error: 4.1705 - val_mean_absolute_percentage_error: 17.3991
Epoch 220/1500
56000/56000 [==============================] - 7s 127us/step - loss: 39.2503 - mean_absolute_error: 4.0636 - mean_absolute_percentage_error: 17.2994 - val_loss: 51.9521 - val_mean_absolute_error: 4.1608 - val_mean_absolute_percentage_error: 17.3510
Epoch 221/1500
56000/56000 [==============================] - 7s 127us/step - loss: 39.6741 - mean_absolute_error: 4.0677 - mean_absolute_percentage_error: 17.3560 - val_loss: 52.7964 - val_mean_absolute_error: 4.2204 - val

...

56000/56000 [==============================] - 7s 126us/step - loss: 27.8009 - mean_absolute_error: 3.6314 - mean_absolute_percentage_error: 15.4835 - val_loss: 51.9338 - val_mean_absolute_error: 3.9829 - val_mean_absolute_percentage_error: 16.3428
Epoch 405/1500
56000/56000 [==============================] - 7s 126us/step - loss: 27.6528 - mean_absolute_error: 3.6321 - mean_absolute_percentage_error: 15.4343 - val_loss: 51.1661 - val_mean_absolute_error: 3.9627 - val_mean_absolute_percentage_error: 16.5577
Epoch 406/1500
56000/56000 [==============================] - 7s 126us/step - loss: 27.6463 - mean_absolute_error: 3.6186 - mean_absolute_percentage_error: 15.4003 - val_loss: 50.6870 - val_mean_absolute_error: 3.9808 - val_mean_absolute_percentage_error: 16.6145
Epoch 407/1500
56000/56000 [==============================] - 7s 126us/step - loss: 30.3865 - mean_absolute_error: 3.7429 - mean_absolute_percentage_error: 15.9873 - val_loss: 52.7807 - val_mean_absolute_error: 4.0180 - val

56000/56000 [==============================] - 7s 126us/step - loss: 27.4617 - mean_absolute_error: 3.5792 - mean_absolute_percentage_error: 15.2419 - val_loss: 52.3324 - val_mean_absolute_error: 3.9617 - val_mean_absolute_percentage_error: 16.6451
Epoch 467/1500
56000/56000 [==============================] - 7s 127us/step - loss: 27.2926 - mean_absolute_error: 3.5630 - mean_absolute_percentage_error: 15.1778 - val_loss: 51.5838 - val_mean_absolute_error: 3.9363 - val_mean_absolute_percentage_error: 16.1423
Epoch 468/1500
56000/56000 [==============================] - 7s 127us/step - loss: 26.9457 - mean_absolute_error: 3.5466 - mean_absolute_percentage_error: 15.0785 - val_loss: 49.8846 - val_mean_absolute_error: 3.9466 - val_mean_absolute_percentage_error: 16.4272
Epoch 469/1500
56000/56000 [==============================] - 7s 126us/step - loss: 27.9910 - mean_absolute_error: 3.6015 - mean_absolute_percentage_error: 15.3195 - val_loss: 50.2169 - val_mean_absolute_error: 3.9761 - val

56000/56000 [==============================] - 7s 127us/step - loss: 25.3336 - mean_absolute_error: 3.4858 - mean_absolute_percentage_error: 14.7734 - val_loss: 50.6694 - val_mean_absolute_error: 3.9174 - val_mean_absolute_percentage_error: 16.1209
Epoch 529/1500
56000/56000 [==============================] - 7s 126us/step - loss: 24.7339 - mean_absolute_error: 3.4637 - mean_absolute_percentage_error: 14.7278 - val_loss: 50.3755 - val_mean_absolute_error: 3.9111 - val_mean_absolute_percentage_error: 16.2803
Epoch 530/1500
56000/56000 [==============================] - 7s 127us/step - loss: 25.6798 - mean_absolute_error: 3.4847 - mean_absolute_percentage_error: 14.8178 - val_loss: 50.8862 - val_mean_absolute_error: 3.9390 - val_mean_absolute_percentage_error: 16.3333
Epoch 531/1500
56000/56000 [==============================] - 7s 127us/step - loss: 25.0652 - mean_absolute_error: 3.4894 - mean_absolute_percentage_error: 14.8598 - val_loss: 50.7090 - val_mean_absolute_error: 3.9239 - val

56000/56000 [==============================] - 7s 127us/step - loss: 25.3407 - mean_absolute_error: 3.4805 - mean_absolute_percentage_error: 14.8018 - val_loss: 50.1571 - val_mean_absolute_error: 3.9174 - val_mean_absolute_percentage_error: 16.1216
Epoch 591/1500
56000/56000 [==============================] - 7s 126us/step - loss: 25.9724 - mean_absolute_error: 3.4742 - mean_absolute_percentage_error: 14.7880 - val_loss: 51.1974 - val_mean_absolute_error: 3.9868 - val_mean_absolute_percentage_error: 16.7937
Epoch 592/1500
56000/56000 [==============================] - 7s 127us/step - loss: 31.3427 - mean_absolute_error: 3.7126 - mean_absolute_percentage_error: 16.1673 - val_loss: 53.8603 - val_mean_absolute_error: 4.3116 - val_mean_absolute_percentage_error: 18.5800
Epoch 593/1500
56000/56000 [==============================] - 7s 126us/step - loss: 34.7722 - mean_absolute_error: 3.8513 - mean_absolute_percentage_error: 16.8831 - val_loss: 52.0083 - val_mean_absolute_error: 4.1427 - val

56000/56000 [==============================] - 7s 127us/step - loss: 22.9355 - mean_absolute_error: 3.3426 - mean_absolute_percentage_error: 14.1764 - val_loss: 49.7290 - val_mean_absolute_error: 3.9026 - val_mean_absolute_percentage_error: 15.9903
Epoch 653/1500
56000/56000 [==============================] - 7s 126us/step - loss: 22.9970 - mean_absolute_error: 3.3462 - mean_absolute_percentage_error: 14.1585 - val_loss: 49.7978 - val_mean_absolute_error: 3.8935 - val_mean_absolute_percentage_error: 15.9630
Epoch 654/1500
56000/56000 [==============================] - 7s 126us/step - loss: 23.0869 - mean_absolute_error: 3.3402 - mean_absolute_percentage_error: 14.1114 - val_loss: 49.4267 - val_mean_absolute_error: 3.8774 - val_mean_absolute_percentage_error: 16.1801
Epoch 655/1500
56000/56000 [==============================] - 7s 127us/step - loss: 22.4032 - mean_absolute_error: 3.3293 - mean_absolute_percentage_error: 14.0883 - val_loss: 49.3987 - val_mean_absolute_error: 3.8795 - val

In [10]:
score=model.evaluate(x_v,y_v)
print("MSE:","%.4f"%score[0]," / MAE:","%.4f"%score[1],"/ MAPE:","%.4f"%score[2])

27545/27545 [==============================] - 12s 438us/step
MSE: 48.6660  / MAE: 4.0091 / MAPE: 16.9589


In [11]:
a=model.predict(x_v)
print("桃竹苗")
print("========MSE========")
print("1hr:%.4f\n"%mse_f(y_v,a,0),"2hrs:%.4f\n"%mse_f(y_v,a,1),"3hrs:%.4f\n"%mse_f(y_v,a,2),"4hrs:%.4f\n"%mse_f(y_v,a,3),"5hrs:%.4f\n"%mse_f(y_v,a,4),"6hrs:%.4f\n"%mse_f(y_v,a,5),"7hrs:%.4f\n"%mse_f(y_v,a,6),"8hrs:%.4f"%mse_f(y_v,a,7))
print("========MAE========")
print("1hr:%.4f\n"%mae_f(y_v,a,0),"2hrs:%.4f\n"%mae_f(y_v,a,1),"3hrs:%.4f\n"%mae_f(y_v,a,2),"4hrs:%.4f\n"%mae_f(y_v,a,3),"5hrs:%.4f\n"%mae_f(y_v,a,4),"6hrs:%.4f\n"%mae_f(y_v,a,5),"7hrs:%.4f\n"%mae_f(y_v,a,6),"8hrs:%.4f"%mae_f(y_v,a,7))
print("========MAPE========")
print("1hr:%.4f\n"%mape_f(y_v,a,0),"2hrs:%.4f\n"%mape_f(y_v,a,1),"3hrs:%.4f\n"%mape_f(y_v,a,2),"4hrs:%.4f\n"%mape_f(y_v,a,3),"5hrs:%.4f\n"%mape_f(y_v,a,4),"6hrs:%.4f\n"%mape_f(y_v,a,5),"7hrs:%.4f\n"%mape_f(y_v,a,6),"8hrs:%.4f"%mape_f(y_v,a,7))
print("========RMSE========")
print("1hr:%.4f\n"%rmse_f(y_v,a,0),"2hrs:%.4f\n"%rmse_f(y_v,a,1),"3hrs:%.4f\n"%rmse_f(y_v,a,2),"4hrs:%.4f\n"%rmse_f(y_v,a,3),"5hrs:%.4f\n"%rmse_f(y_v,a,4),"6hrs:%.4f\n"%rmse_f(y_v,a,5),"7hrs:%.4f\n"%rmse_f(y_v,a,6),"8hrs:%.4f"%rmse_f(y_v,a,7))


桃竹苗
========MSE========
1hr:32.0281
 2hrs:46.8958
 3hrs:46.5322
 4hrs:43.3821
 5hrs:60.5769
 6hrs:51.0170
 7hrs:55.3974
 8hrs:53.4981
========MAE========
1hr:3.3900
 2hrs:3.6326
 3hrs:3.8664
 4hrs:4.0150
 5hrs:4.1712
 6hrs:4.2549
 7hrs:4.3037
 8hrs:4.4390
========MAPE========
1hr:0.1423
 2hrs:0.1516
 3hrs:0.1616
 4hrs:0.1713
 5hrs:0.1767
 6hrs:0.1776
 7hrs:0.1832
 8hrs:0.1925
========RMSE========
1hr:5.6593
 2hrs:6.8481
 3hrs:6.8215
 4hrs:6.5865
 5hrs:7.7831
 6hrs:7.1426
 7hrs:7.4429
 8hrs:7.3142


In [12]:
model.save('D:/download/finalmodel/lstmmodel4.h5')


In [13]:
#data_loc= pd.read_csv('D:/download/完成/final1.csv', encoding='utf-8')
#data_loc=data[data['location']==0]

data_loc=data_loc.dropna()
#data_loc=data_loc.drop(columns=['location','deviceId'])
validation_sample=80000
data_loc = shuffle(data_loc) #資料亂序

#col_name = data_loc.columns.tolist()
#for col in col_name:
    #drop_id = data_loc[data_loc[col]>=100].index 
    #data_loc=data_loc.drop(drop_id)
    #drop_id = data_loc[data_loc[col]==0].index 
    #data_loc=data_loc.drop(drop_id)

#=============資料處理===============#
loc_0_x=data_loc.iloc[:,:24]
loc_0_x=loc_0_x.values
StandardScaler_scaler = preprocessing.StandardScaler()
StandardScaler_scaler = StandardScaler_scaler.fit(loc_0_x) #標準化模型
loc_0_x = StandardScaler_scaler.transform(loc_0_x)
joblib.dump(StandardScaler_scaler, 'scaler4_8.pkl')


loc_0_x=loc_0_x.reshape(len(loc_0_x),8,3) #作三維資料

loc_0_y=data_loc.iloc[:,24:].values

#=============測試集==============#
x_v=loc_0_x[validation_sample:]
y_v=loc_0_y[validation_sample:]

#=============訓練集資料===============#
x=loc_0_x[:validation_sample]
y=loc_0_y[:validation_sample]

#隨機分訓練集及測試集

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

  

#建立模型
input_data=Input(shape=(8,3))
m=GRU(256,return_sequences=True)(input_data)
m=Dropout(0.3)(m)
m=GRU(256)(m)
m=Dropout(0.3)(m)
#m=LSTM(256)(m)
#m=Dropout(0.2)(m)

m=Dense(256,activation='relu')(m)
m=Dropout(0.3)(m)
m=Dense(256,activation='relu')(m)
m=Dropout(0.3)(m)
m=Dense(256,activation='relu')(m)
m=Dropout(0.3)(m)
output=Dense(8,activation=None)(m)

model = Model(inputs=input_data,outputs=output)
callback = EarlyStopping(monitor="val_loss", patience=100, verbose=1, mode="auto")
model.compile(optimizer='adam',loss=['mse'],metrics=['mae','mape'])
result = model.fit(X_train,y_train,epochs=1500 ,batch_size=2400,validation_data=(X_test, y_test),callbacks=[callback])

Train on 56000 samples, validate on 24000 samples
Epoch 1/1500
56000/56000 [==============================] - 5s 98us/step - loss: 518.4992 - mean_absolute_error: 18.2757 - mean_absolute_percentage_error: 73.7495 - val_loss: 199.9017 - val_mean_absolute_error: 10.1517 - val_mean_absolute_percentage_error: 42.8992
Epoch 2/1500
56000/56000 [==============================] - 3s 59us/step - loss: 180.9910 - mean_absolute_error: 9.8571 - mean_absolute_percentage_error: 43.5803 - val_loss: 146.2135 - val_mean_absolute_error: 8.4321 - val_mean_absolute_percentage_error: 36.1143
Epoch 3/1500
56000/56000 [==============================] - 3s 59us/step - loss: 151.2826 - mean_absolute_error: 8.9779 - mean_absolute_percentage_error: 41.2306 - val_loss: 126.0085 - val_mean_absolute_error: 7.8009 - val_mean_absolute_percentage_error: 33.7361
Epoch 4/1500
56000/56000 [==============================] - 3s 59us/step - loss: 133.7512 - mean_absolute_error: 8.3012 - mean_absolute_percentage_error: 37.82

Epoch 32/1500
56000/56000 [==============================] - 4s 65us/step - loss: 81.2811 - mean_absolute_error: 6.1103 - mean_absolute_percentage_error: 26.9848 - val_loss: 92.1331 - val_mean_absolute_error: 6.4158 - val_mean_absolute_percentage_error: 23.8886
Epoch 33/1500
56000/56000 [==============================] - 4s 65us/step - loss: 80.5335 - mean_absolute_error: 6.0770 - mean_absolute_percentage_error: 26.7223 - val_loss: 89.4697 - val_mean_absolute_error: 6.2546 - val_mean_absolute_percentage_error: 23.5053
Epoch 34/1500
56000/56000 [==============================] - 4s 65us/step - loss: 79.7480 - mean_absolute_error: 6.0422 - mean_absolute_percentage_error: 26.6308 - val_loss: 94.7004 - val_mean_absolute_error: 6.5151 - val_mean_absolute_percentage_error: 23.9385
Epoch 35/1500
56000/56000 [==============================] - 4s 64us/step - loss: 80.1984 - mean_absolute_error: 6.0522 - mean_absolute_percentage_error: 26.5957 - val_loss: 99.1188 - val_mean_absolute_error: 6.753

56000/56000 [==============================] - 4s 67us/step - loss: 55.9249 - mean_absolute_error: 4.8247 - mean_absolute_percentage_error: 20.8513 - val_loss: 63.3026 - val_mean_absolute_error: 4.9504 - val_mean_absolute_percentage_error: 18.8251
Epoch 95/1500
56000/56000 [==============================] - 4s 67us/step - loss: 55.6848 - mean_absolute_error: 4.8074 - mean_absolute_percentage_error: 20.7927 - val_loss: 65.1877 - val_mean_absolute_error: 5.0441 - val_mean_absolute_percentage_error: 19.1793
Epoch 96/1500
56000/56000 [==============================] - 4s 67us/step - loss: 55.6122 - mean_absolute_error: 4.8076 - mean_absolute_percentage_error: 20.7307 - val_loss: 61.1808 - val_mean_absolute_error: 4.8129 - val_mean_absolute_percentage_error: 18.6064
Epoch 97/1500
56000/56000 [==============================] - 4s 67us/step - loss: 55.1518 - mean_absolute_error: 4.7807 - mean_absolute_percentage_error: 20.6752 - val_loss: 59.3874 - val_mean_absolute_error: 4.7400 - val_mean_a

56000/56000 [==============================] - 4s 68us/step - loss: 44.7213 - mean_absolute_error: 4.2577 - mean_absolute_percentage_error: 18.2238 - val_loss: 51.9794 - val_mean_absolute_error: 4.3199 - val_mean_absolute_percentage_error: 17.6924
Epoch 157/1500
56000/56000 [==============================] - 4s 67us/step - loss: 44.6464 - mean_absolute_error: 4.2629 - mean_absolute_percentage_error: 18.2113 - val_loss: 49.1457 - val_mean_absolute_error: 4.1908 - val_mean_absolute_percentage_error: 17.8909
Epoch 158/1500
56000/56000 [==============================] - 4s 67us/step - loss: 44.7754 - mean_absolute_error: 4.2656 - mean_absolute_percentage_error: 18.2056 - val_loss: 49.8868 - val_mean_absolute_error: 4.2046 - val_mean_absolute_percentage_error: 17.7431
Epoch 159/1500
56000/56000 [==============================] - 4s 68us/step - loss: 44.3468 - mean_absolute_error: 4.2619 - mean_absolute_percentage_error: 18.2386 - val_loss: 50.8393 - val_mean_absolute_error: 4.2655 - val_mea

56000/56000 [==============================] - 4s 68us/step - loss: 37.1829 - mean_absolute_error: 3.9737 - mean_absolute_percentage_error: 16.9765 - val_loss: 46.5071 - val_mean_absolute_error: 4.0244 - val_mean_absolute_percentage_error: 17.8150
Epoch 219/1500
56000/56000 [==============================] - 4s 68us/step - loss: 37.3800 - mean_absolute_error: 3.9799 - mean_absolute_percentage_error: 17.0035 - val_loss: 49.1271 - val_mean_absolute_error: 4.1641 - val_mean_absolute_percentage_error: 17.9640
Epoch 220/1500
56000/56000 [==============================] - 4s 68us/step - loss: 37.2657 - mean_absolute_error: 3.9831 - mean_absolute_percentage_error: 17.0269 - val_loss: 48.9655 - val_mean_absolute_error: 4.1449 - val_mean_absolute_percentage_error: 18.2134
Epoch 221/1500
56000/56000 [==============================] - 4s 68us/step - loss: 37.9707 - mean_absolute_error: 3.9937 - mean_absolute_percentage_error: 17.0996 - val_loss: 48.3829 - val_mean_absolute_error: 4.0806 - val_mea

...

56000/56000 [==============================] - 4s 69us/step - loss: 27.4801 - mean_absolute_error: 3.6086 - mean_absolute_percentage_error: 15.3821 - val_loss: 46.7717 - val_mean_absolute_error: 3.9108 - val_mean_absolute_percentage_error: 16.1800
Epoch 405/1500
56000/56000 [==============================] - 4s 68us/step - loss: 27.3142 - mean_absolute_error: 3.5899 - mean_absolute_percentage_error: 15.3027 - val_loss: 46.6940 - val_mean_absolute_error: 3.9503 - val_mean_absolute_percentage_error: 16.2816
Epoch 406/1500
56000/56000 [==============================] - 4s 68us/step - loss: 27.5180 - mean_absolute_error: 3.5937 - mean_absolute_percentage_error: 15.3319 - val_loss: 45.9029 - val_mean_absolute_error: 3.8825 - val_mean_absolute_percentage_error: 16.3518
Epoch 407/1500
56000/56000 [==============================] - 4s 68us/step - loss: 27.4947 - mean_absolute_error: 3.6012 - mean_absolute_percentage_error: 15.3868 - val_loss: 47.5338 - val_mean_absolute_error: 3.9634 - val_mea

56000/56000 [==============================] - 4s 68us/step - loss: 25.8599 - mean_absolute_error: 3.5169 - mean_absolute_percentage_error: 14.9918 - val_loss: 47.3004 - val_mean_absolute_error: 3.9300 - val_mean_absolute_percentage_error: 15.8515
Epoch 467/1500
56000/56000 [==============================] - 4s 68us/step - loss: 26.4413 - mean_absolute_error: 3.5495 - mean_absolute_percentage_error: 15.1556 - val_loss: 45.3035 - val_mean_absolute_error: 3.8514 - val_mean_absolute_percentage_error: 16.4918
Epoch 468/1500
56000/56000 [==============================] - 4s 68us/step - loss: 25.7488 - mean_absolute_error: 3.5342 - mean_absolute_percentage_error: 15.0530 - val_loss: 44.8912 - val_mean_absolute_error: 3.8291 - val_mean_absolute_percentage_error: 16.2017
Epoch 469/1500
56000/56000 [==============================] - 4s 68us/step - loss: 25.6398 - mean_absolute_error: 3.5030 - mean_absolute_percentage_error: 14.9124 - val_loss: 45.5857 - val_mean_absolute_error: 3.8342 - val_mea

56000/56000 [==============================] - 4s 68us/step - loss: 26.2630 - mean_absolute_error: 3.5036 - mean_absolute_percentage_error: 14.9773 - val_loss: 47.4647 - val_mean_absolute_error: 3.8743 - val_mean_absolute_percentage_error: 15.6056
Epoch 529/1500
56000/56000 [==============================] - 4s 68us/step - loss: 26.0473 - mean_absolute_error: 3.5030 - mean_absolute_percentage_error: 14.9151 - val_loss: 45.1217 - val_mean_absolute_error: 3.8333 - val_mean_absolute_percentage_error: 15.7627
Epoch 530/1500
56000/56000 [==============================] - 4s 68us/step - loss: 26.0509 - mean_absolute_error: 3.4751 - mean_absolute_percentage_error: 14.8248 - val_loss: 44.7709 - val_mean_absolute_error: 3.7955 - val_mean_absolute_percentage_error: 15.7882
Epoch 531/1500
56000/56000 [==============================] - 4s 68us/step - loss: 25.1567 - mean_absolute_error: 3.4681 - mean_absolute_percentage_error: 14.7950 - val_loss: 44.7180 - val_mean_absolute_error: 3.7997 - val_mea

56000/56000 [==============================] - 4s 69us/step - loss: 23.6064 - mean_absolute_error: 3.3837 - mean_absolute_percentage_error: 14.3883 - val_loss: 43.9922 - val_mean_absolute_error: 3.8021 - val_mean_absolute_percentage_error: 15.8125
Epoch 591/1500
56000/56000 [==============================] - 4s 68us/step - loss: 23.7852 - mean_absolute_error: 3.3928 - mean_absolute_percentage_error: 14.4074 - val_loss: 43.3978 - val_mean_absolute_error: 3.7570 - val_mean_absolute_percentage_error: 15.6784
Epoch 592/1500
56000/56000 [==============================] - 4s 68us/step - loss: 23.4817 - mean_absolute_error: 3.3811 - mean_absolute_percentage_error: 14.4033 - val_loss: 43.9849 - val_mean_absolute_error: 3.7987 - val_mean_absolute_percentage_error: 15.7397
Epoch 593/1500
56000/56000 [==============================] - 4s 68us/step - loss: 23.4462 - mean_absolute_error: 3.3821 - mean_absolute_percentage_error: 14.4145 - val_loss: 43.4044 - val_mean_absolute_error: 3.7709 - val_mea

56000/56000 [==============================] - 4s 68us/step - loss: 23.2237 - mean_absolute_error: 3.3352 - mean_absolute_percentage_error: 14.1934 - val_loss: 44.4168 - val_mean_absolute_error: 3.7462 - val_mean_absolute_percentage_error: 15.5373
Epoch 653/1500
56000/56000 [==============================] - 4s 68us/step - loss: 23.1582 - mean_absolute_error: 3.3414 - mean_absolute_percentage_error: 14.1842 - val_loss: 44.5217 - val_mean_absolute_error: 3.7964 - val_mean_absolute_percentage_error: 16.0142
Epoch 654/1500
56000/56000 [==============================] - 4s 68us/step - loss: 22.9953 - mean_absolute_error: 3.3471 - mean_absolute_percentage_error: 14.2300 - val_loss: 45.0335 - val_mean_absolute_error: 3.8078 - val_mean_absolute_percentage_error: 15.6108
Epoch 655/1500
56000/56000 [==============================] - 4s 68us/step - loss: 22.8223 - mean_absolute_error: 3.3291 - mean_absolute_percentage_error: 14.1698 - val_loss: 44.1918 - val_mean_absolute_error: 3.7682 - val_mea

In [14]:
score=model.evaluate(x_v,y_v)
print("MSE:","%.4f"%score[0]," / MAE:","%.4f"%score[1],"/ MAPE:","%.4f"%score[2])

27545/27545 [==============================] - 24s 877us/step
MSE: 41.6545  / MAE: 3.8212 / MAPE: 15.9021


In [15]:
a=model.predict(x_v)
print("桃竹苗")
print("========MSE========")
print("1hr:%.4f\n"%mse_f(y_v,a,0),"2hrs:%.4f\n"%mse_f(y_v,a,1),"3hrs:%.4f\n"%mse_f(y_v,a,2),"4hrs:%.4f\n"%mse_f(y_v,a,3),"5hrs:%.4f\n"%mse_f(y_v,a,4),"6hrs:%.4f\n"%mse_f(y_v,a,5),"7hrs:%.4f\n"%mse_f(y_v,a,6),"8hrs:%.4f"%mse_f(y_v,a,7))
print("========MAE========")
print("1hr:%.4f\n"%mae_f(y_v,a,0),"2hrs:%.4f\n"%mae_f(y_v,a,1),"3hrs:%.4f\n"%mae_f(y_v,a,2),"4hrs:%.4f\n"%mae_f(y_v,a,3),"5hrs:%.4f\n"%mae_f(y_v,a,4),"6hrs:%.4f\n"%mae_f(y_v,a,5),"7hrs:%.4f\n"%mae_f(y_v,a,6),"8hrs:%.4f"%mae_f(y_v,a,7))
print("========MAPE========")
print("1hr:%.4f\n"%mape_f(y_v,a,0),"2hrs:%.4f\n"%mape_f(y_v,a,1),"3hrs:%.4f\n"%mape_f(y_v,a,2),"4hrs:%.4f\n"%mape_f(y_v,a,3),"5hrs:%.4f\n"%mape_f(y_v,a,4),"6hrs:%.4f\n"%mape_f(y_v,a,5),"7hrs:%.4f\n"%mape_f(y_v,a,6),"8hrs:%.4f"%mape_f(y_v,a,7))
print("========RMSE========")
print("1hr:%.4f\n"%rmse_f(y_v,a,0),"2hrs:%.4f\n"%rmse_f(y_v,a,1),"3hrs:%.4f\n"%rmse_f(y_v,a,2),"4hrs:%.4f\n"%rmse_f(y_v,a,3),"5hrs:%.4f\n"%rmse_f(y_v,a,4),"6hrs:%.4f\n"%rmse_f(y_v,a,5),"7hrs:%.4f\n"%rmse_f(y_v,a,6),"8hrs:%.4f"%rmse_f(y_v,a,7))


桃竹苗
========MSE========
1hr:34.3502
 2hrs:47.4905
 3hrs:41.6596
 4hrs:40.9325
 5hrs:44.4760
 6hrs:36.5251
 7hrs:39.2565
 8hrs:48.5456
========MAE========
1hr:3.3709
 2hrs:3.5831
 3hrs:3.7105
 4hrs:3.8011
 5hrs:3.8953
 6hrs:3.9188
 7hrs:4.0481
 8hrs:4.2416
========MAPE========
1hr:0.1391
 2hrs:0.1462
 3hrs:0.1524
 4hrs:0.1581
 5hrs:0.1607
 6hrs:0.1650
 7hrs:0.1701
 8hrs:0.1806
========RMSE========
1hr:5.8609
 2hrs:6.8913
 3hrs:6.4544
 4hrs:6.3979
 5hrs:6.6690
 6hrs:6.0436
 7hrs:6.2655
 8hrs:6.9675


In [16]:
model.save('D:/download/finalmodel/GRUmodel4.h5')
